# Large Ensemble with Same Architecture (Variance Reduction)

This experiment focuses on variance reduction by:
1. Using the PROVEN exp_003 architecture (Combined Spange + DRFP + Arrhenius)
2. Increasing ensemble from 5 to 15 models with SAME architecture
3. Using different random seeds for each model
4. Averaging predictions

Goal: Reduce CV-LB gap through more stable predictions.

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set base seed
np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')

Using device: cuda
GPU: NVIDIA A100-SXM4-80GB


In [2]:
# Load data and features
DATA_PATH = '/home/data'

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT NAME"]]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

# Load feature lookups
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)

# Filter DRFP to high-variance features (122 features)
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'Spange shape: {SPANGE_DF.shape}')
print(f'DRFP filtered shape: {DRFP_FILTERED.shape}')

Spange shape: (26, 13)
DRFP filtered shape: (24, 122)


In [3]:
# Combined Featurizer (SAME as exp_003)
class CombinedFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.drfp_df = DRFP_FILTERED
        self.feats_dim = 2 + 3 + self.spange_df.shape[1] + self.drfp_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[["Residence Time", "Temperature"]].values.astype(np.float64)
        
        # Arrhenius kinetics features
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            
            pct = X["SolventB%"].values.reshape(-1, 1)
            
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
                X_drfp = B_drfp * (1 - (1-pct)) + A_drfp * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
                X_drfp = A_drfp * (1 - pct) + B_drfp * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
        
        return torch.tensor(np.hstack([X_kinetic, X_spange, X_drfp]))

print(f'Feature dimension: {CombinedFeaturizer().feats_dim}')

Feature dimension: 140


In [4]:
# Simple MLP (SAME architecture as exp_003 - proven to work)
class MLPInternal(nn.Module):
    def __init__(self, input_dim, hidden_dims=[256, 128, 64], output_dim=3, dropout=0.3):
        super(MLPInternal, self).__init__()
        layers = [nn.BatchNorm1d(input_dim)]
        
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, h_dim),
                nn.BatchNorm1d(h_dim),
                nn.ReLU(),
                nn.Dropout(dropout)
            ])
            prev_dim = h_dim
        
        layers.append(nn.Linear(prev_dim, output_dim))
        layers.append(nn.Sigmoid())
        
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

# Large Ensemble Model - 15 models with SAME architecture, different seeds
class LargeEnsembleMLP(nn.Module):
    def __init__(self, data='single', n_models=15):
        super().__init__()
        self.data_type = data
        self.n_models = n_models
        self.featurizer = CombinedFeaturizer(mixed=(data=='full'))
        self.models = nn.ModuleList()

    def train_model(self, X_train, y_train, epochs=300, batch_size=32, lr=5e-4):
        X_std = self.featurizer.featurize(X_train, flip=False)
        y_vals = torch.tensor(y_train.values)
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize(X_train, flip=True)
            X_all = torch.cat([X_std, X_flip], dim=0)
            y_all = torch.cat([y_vals, y_vals], dim=0)
        else:
            X_all = X_std
            y_all = y_vals
            
        input_dim = X_all.shape[1]
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        for i in range(self.n_models):
            # Different seed for each model
            torch.manual_seed(42 + i * 13)
            np.random.seed(42 + i * 13)
            
            # SAME architecture for all models (proven to work)
            model = MLPInternal(input_dim, hidden_dims=[256, 128, 64], dropout=0.3).to(device)
            model.train()
            self.models.append(model)
            
            dataset = TensorDataset(X_all, y_all)
            loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            
            optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
            criterion = nn.HuberLoss()
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, mode='min', factor=0.5, patience=20
            )
            
            for epoch in range(epochs):
                epoch_loss = 0.0
                for inputs, targets in loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                    optimizer.step()
                    epoch_loss += loss.item() * inputs.size(0)
                
                scheduler.step(epoch_loss / len(dataset))

    def predict(self, X):
        device = next(self.models[0].parameters()).device
        
        if self.data_type == 'full':
            X_std = self.featurizer.featurize(X, flip=False).to(device)
            X_flip = self.featurizer.featurize(X, flip=True).to(device)
            
            pred_sum = torch.zeros((len(X), 3)).to(device)
            
            with torch.no_grad():
                for model in self.models:
                    model.eval()
                    p1 = model(X_std)
                    p2 = model(X_flip)
                    pred_sum += (p1 + p2) * 0.5
            
            avg_pred = pred_sum / self.n_models
        else:
            X_std = self.featurizer.featurize(X).to(device)
            pred_sum = torch.zeros((len(X), 3)).to(device)
            with torch.no_grad():
                for model in self.models:
                    model.eval()
                    pred_sum += model(X_std)
            avg_pred = pred_sum / self.n_models

        return avg_pred.cpu()

print('LargeEnsembleMLP defined (15 models, same architecture)')

LargeEnsembleMLP defined (15 models, same architecture)


In [5]:
# Run cross-validation for single solvent data
print("\n--- TASK 0: Single Solvent (24 folds) ---")
X, Y = load_data("single_solvent")
split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []
all_actuals = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=24):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = LargeEnsembleMLP(data='single', n_models=15)
    model.train_model(train_X, train_Y, epochs=300)
    predictions = model.predict(test_X).numpy()
    
    for row_idx, row in enumerate(predictions):
        all_predictions.append({
            "task": 0, "fold": fold_idx, "row": row_idx,
            "target_1": row[0], "target_2": row[1], "target_3": row[2]
        })
    
    all_actuals.append(test_Y.values)

submission_single = pd.DataFrame(all_predictions)

actuals_single = np.vstack(all_actuals)
preds_single = submission_single[['target_1', 'target_2', 'target_3']].values
mse_single = np.mean((actuals_single - preds_single) ** 2)
print(f'\nSingle Solvent MSE: {mse_single:.6f}')


--- TASK 0: Single Solvent (24 folds) ---


  0%|          | 0/24 [00:00<?, ?it/s]

  4%|▍         | 1/24 [05:44<2:12:03, 344.49s/it]

  8%|▊         | 2/24 [11:18<2:04:06, 338.49s/it]

 12%|█▎        | 3/24 [16:39<1:55:37, 330.36s/it]

...

 92%|█████████▏| 22/24 [2:10:23<12:32, 376.35s/it]

 96%|█████████▌| 23/24 [2:17:12<06:26, 386.17s/it]

100%|██████████| 24/24 [2:23:11<00:00, 378.14s/it]

100%|██████████| 24/24 [2:23:11<00:00, 358.00s/it]


Single Solvent MSE: 0.011533


In [6]:
# Run cross-validation for full data (mixtures)
print("\n--- TASK 1: Full Data (13 folds) ---")
X, Y = load_data("full")
split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions_full = []
all_actuals_full = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=13):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = LargeEnsembleMLP(data='full', n_models=15)
    model.train_model(train_X, train_Y, epochs=300)
    predictions = model.predict(test_X).numpy()
    
    for row_idx, row in enumerate(predictions):
        all_predictions_full.append({
            "task": 1, "fold": fold_idx, "row": row_idx,
            "target_1": row[0], "target_2": row[1], "target_3": row[2]
        })
    
    all_actuals_full.append(test_Y.values)

submission_full = pd.DataFrame(all_predictions_full)

actuals_full = np.vstack(all_actuals_full)
preds_full = submission_full[['target_1', 'target_2', 'target_3']].values
mse_full = np.mean((actuals_full - preds_full) ** 2)
print(f'\nFull Data MSE: {mse_full:.6f}')


--- TASK 1: Full Data (13 folds) ---


  0%|          | 0/13 [00:00<?, ?it/s]

  8%|▊         | 1/13 [19:41<3:56:17, 1181.48s/it]

 15%|█▌        | 2/13 [37:20<3:23:23, 1109.36s/it]

 23%|██▎       | 3/13 [55:06<3:01:35, 1089.56s/it]

...

 85%|████████▍ | 11/13 [3:24:06<36:48, 1104.15s/it]

 92%|█████████▏| 12/13 [3:41:47<18:11, 1091.27s/it]

100%|██████████| 13/13 [3:59:35<00:00, 1084.19s/it]

100%|██████████| 13/13 [3:59:35<00:00, 1105.84s/it]


Full Data MSE: 0.009841


In [7]:
# Calculate overall MSE
n_single = len(actuals_single)
n_full = len(actuals_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n=== FINAL RESULTS ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')
print(f'\nComparison to exp_003 (5 models):')
print(f'  exp_003 CV: 0.010501')
print(f'  This (15 models): {overall_mse:.6f}')
print(f'  Change: {(overall_mse - 0.010501) / 0.010501 * 100:+.1f}%')


=== FINAL RESULTS ===
Single Solvent MSE: 0.011533 (n=656)
Full Data MSE: 0.009841 (n=1227)
Overall MSE: 0.010430

Comparison to exp_003 (5 models):
  exp_003 CV: 0.010501
  This (15 models): 0.010430
  Change: -0.7%


In [8]:
# Create submission file
submission = pd.concat([submission_single, submission_full])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)
print(f'Submission saved to /home/submission/submission.csv')
print(f'Submission shape: {submission.shape}')
print(submission.head())

Submission saved to /home/submission/submission.csv
Submission shape: (1883, 7)
    index  task  fold  row  target_1  target_2  target_3
id                                                      
0       0     0     0    0  0.017626  0.017251  0.928387
1       1     0     0    1  0.025680  0.024549  0.903876
2       2     0     0    2  0.037863  0.034964  0.860421
3       3     0     0    3  0.053891  0.047299  0.801697
4       4     0     0    4  0.073909  0.061877  0.732290
